# Deepfake Detection Pipeline

This notebook demonstrates the complete deepfake detection pipeline:
1. Video frame extraction
2. Face detection and cropping
3. Multi-detector analysis (CNN, Temporal, Lip-Sync, Frequency)
4. Ensemble fusion
5. Result visualization


## 1. Setup and Imports


In [4]:
import os
import json
import uuid
from pathlib import Path
from typing import Dict, Any, List, Optional
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display, HTML

# Import project modules
import backend.utils.file_utils as file_utils
import backend.utils.video_utils as video_utils
import backend.utils.face_utils as face_utils
import backend.utils.mouth_cropper as mouth_cropper
import backend.utils.temporal_utils as temporal_utils
import backend.utils.aggregation as aggregation
import backend.utils.ensemble as ensemble
from backend.utils.model_cache import model_cache
from backend.utils.abnormality_analyzer import AbnormalityAnalyzer
from backend.utils.technique_identifier import TechniqueIdentifier

print("✓ All imports successful")


ModuleNotFoundError: No module named 'matplotlib'

## 2. Initialize Models


In [ ]:
# Initialize model cache (pre-loads all models)
print("Initializing models...")
model_cache.initialize()
print("✓ Models initialized")

# Get detectors
cnn_detector = model_cache.get_cnn_detector()
temporal_detector = model_cache.get_temporal_detector()
lipsync_detector = model_cache.get_lipsync_detector()
freq_detector = model_cache.get_frequency_detector()

print("✓ All detectors loaded")


## 3. Load Video

**Note:** Update the path to your video file


In [ ]:
# Automatically find a video file in storage/uploads
# You can also specify a custom path: video_path = "storage/uploads/your_video.mp4"

uploads_dir = Path("storage/uploads")
video_path = None

# Try to find an MP4 file
if uploads_dir.exists():
    video_files = list(uploads_dir.glob("*.mp4"))
    if video_files:
        # Use the first available video
        video_path = str(video_files[0])
        print(f"✓ Found video: {video_path}")
        file_size = Path(video_path).stat().st_size / (1024 * 1024)  # MB
        print(f"  File size: {file_size:.2f} MB")
        print(f"  Total videos available: {len(video_files)}")
    else:
        print(f"⚠ No MP4 files found in {uploads_dir}")
        print("Please add a video file to storage/uploads/")
else:
    print(f"⚠ Directory not found: {uploads_dir}")
    print("Please create the directory and add a video file")

# If no video found, set a placeholder (will skip processing)
if video_path is None:
    video_path = "storage/uploads/your_video.mp4"
    print(f"\n⚠ Using placeholder path: {video_path}")
    print("Update this cell with a valid video path to run the analysis")


## 4. Extract Frames


In [ ]:
# Check if video exists before processing
if not Path(video_path).exists() or "your_video.mp4" in video_path:
    print("⚠ Skipping frame extraction - no valid video file")
    print("Please update the video_path in the previous cell")
    frame_count = 0
    job_id = None
else:
    # Generate unique job ID
    job_id = str(uuid.uuid4())
    print(f"Job ID: {job_id}")
    
    # Setup directories
    base_dir = os.path.abspath("storage")
    frames_output_dir = os.path.normpath(os.path.join(base_dir, "frames", job_id))
    
    # Extract frames (1 frame per second)
    print("\nExtracting frames...")
    try:
        frame_count = video_utils.extract_frames(
            video_path=video_path,
            output_dir=frames_output_dir,
            fps=1
        )
        print(f"✓ Extracted {frame_count} frames")
        
        # Display first frame
        if frame_count > 0:
            from PIL import Image
            first_frame = list(Path(frames_output_dir).glob("frame_*.jpg"))[0]
            img = Image.open(first_frame)
            plt.figure(figsize=(10, 6))
            plt.imshow(img)
            plt.title(f"First Frame (Total: {frame_count} frames)")
            plt.axis('off')
            plt.show()
    except Exception as e:
        print(f"⚠ Error extracting frames: {e}")
        frame_count = 0
        import traceback
        traceback.print_exc()


## 5. Extract Faces


In [ ]:
# Extract faces from frames
if job_id is None or frame_count == 0:
    print("⚠ Skipping face extraction - no frames available")
    face_count = 0
else:
    faces_output_dir = os.path.normpath(os.path.join(base_dir, "faces", job_id))
    
    print("Extracting faces...")
    try:
        face_count = face_utils.extract_faces_from_frames(
            frames_dir=frames_output_dir,
            output_dir=faces_output_dir
        )
        print(f"✓ Extracted {face_count} faces")
    except Exception as e:
        print(f"⚠ Error extracting faces: {e}")
        face_count = 0
        import traceback
        traceback.print_exc()

# Display sample faces
if face_count > 0:
    face_files = sorted(Path(faces_output_dir).glob("face_*.jpg"))[:6]  # Show first 6
    
    fig, axes = plt.subplots(2, 3, figsize=(12, 8))
    axes = axes.flatten()
    
    for idx, face_path in enumerate(face_files):
        img = Image.open(face_path)
        axes[idx].imshow(img)
        axes[idx].set_title(f"Face {idx+1}")
        axes[idx].axis('off')
    
    # Hide unused subplots
    for idx in range(len(face_files), len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()


## 6. Run CNN Detection


In [ ]:
# Run CNN detection on all faces
# Initialize variables if not set
if 'face_count' not in locals():
    face_count = 0
if 'job_id' not in locals():
    job_id = None
if 'base_dir' not in locals():
    base_dir = os.path.abspath("storage")
if 'faces_output_dir' not in locals() and job_id:
    faces_output_dir = os.path.normpath(os.path.join(base_dir, "faces", job_id))
else:
    faces_output_dir = None

detections: List[Dict[str, Any]] = []

if face_count > 0 and faces_output_dir and Path(faces_output_dir).exists():
    print("Running CNN detection...")
    face_files = sorted(Path(faces_output_dir).glob("face_*.jpg"))
    face_paths = [str(f) for f in face_files]
    
    # Use batch prediction if available (much faster)
    if hasattr(cnn_detector, 'predict_batch'):
        print(f"Using batch processing for {len(face_paths)} faces")
        fake_scores = cnn_detector.predict_batch(face_paths)
    else:
        print(f"Using individual predictions for {len(face_paths)} faces")
        fake_scores = [cnn_detector.predict(path) for path in face_paths]
    
    # Create detections list
    for idx, (face_path, fake_score) in enumerate(zip(face_files, fake_scores)):
        detections.append({
            "face_file": face_path.name,
            "frame": idx + 1,
            "fake_score": round(fake_score, 4)
        })
    
    # Sort by highest fake_score (most suspicious first)
    detections.sort(key=lambda x: x["fake_score"], reverse=True)
    
    print(f"✓ Processed {len(detections)} faces")
    print(f"  Max score: {max(fake_scores):.4f}")
    print(f"  Mean score: {np.mean(fake_scores):.4f}")
    
    # Visualize scores
    scores = [d["fake_score"] for d in detections]
    plt.figure(figsize=(10, 5))
    plt.hist(scores, bins=20, edgecolor='black', alpha=0.7)
    plt.xlabel('Fake Score')
    plt.ylabel('Frequency')
    plt.title('Distribution of CNN Fake Scores')
    plt.axvline(x=0.5, color='r', linestyle='--', label='Threshold (0.5)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
else:
    print("⚠ No faces detected")


## 7. Run Frequency Detection


In [ ]:
# Compute frequency scores for all faces
# Initialize variables if not set
if 'job_id' not in locals() or job_id is None:
    job_id = str(uuid.uuid4()) if 'uuid' in dir() else "test"
if 'results_dir' not in locals():
    results_dir = os.path.abspath("results")
if 'faces_output_dir' not in locals():
    faces_output_dir = None

frequency_debug_dir = os.path.normpath(os.path.join(results_dir, job_id, "frequency_maps"))
freq_scores_dict = {}

if face_count > 0 and faces_output_dir and Path(faces_output_dir).exists():
    print("Running frequency detection...")
    freq_scores_dict = freq_detector.batch_compute(
        faces_dir=faces_output_dir,
        output_debug_dir=frequency_debug_dir
    )
    
    # Add frequency scores to detections
    for detection in detections:
        face_filename = detection["face_file"]
        freq_score = freq_scores_dict.get(face_filename, 0.5)
        detection["freq_score"] = round(freq_score, 4)
    
    freq_scores = list(freq_scores_dict.values())
    print(f"✓ Computed frequency scores for {len(freq_scores)} faces")
    print(f"  Mean frequency score: {np.mean(freq_scores):.4f}")
    print(f"  Max frequency score: {max(freq_scores):.4f}")
else:
    print("⚠ No faces to analyze")


## 8. Extract Mouth Regions and Run Lip-Sync Analysis


In [ ]:
# Extract mouth regions
# Initialize variables if not set
if 'job_id' not in locals() or job_id is None:
    job_id = str(uuid.uuid4()) if 'uuid' in dir() else "test"
if 'base_dir' not in locals():
    base_dir = os.path.abspath("storage")
if 'face_count' not in locals():
    face_count = 0
if 'faces_output_dir' not in locals():
    faces_output_dir = None

mouth_output_dir = os.path.normpath(os.path.join(base_dir, "mouth", job_id))
mouth_count = 0
lip_sync_score = None

if face_count > 0 and faces_output_dir and Path(faces_output_dir).exists():
    print("Extracting mouth regions...")
    mouth_count = mouth_cropper.extract_mouth_frames(
        faces_dir=faces_output_dir,
        output_dir=mouth_output_dir
    )
    print(f"✓ Extracted {mouth_count} mouth regions")
    
    # Extract audio from video
    audio_output_path = os.path.normpath(os.path.join(base_dir, "audio", f"{job_id}.wav"))
    
    try:
        print("\nExtracting audio...")
        lipsync_detector.extract_audio(video_path=video_path, out_wav=audio_output_path)
        print(f"✓ Audio extracted to: {audio_output_path}")
        
        # Compute lip-sync score
        print("\nComputing lip-sync score...")
        lip_sync_score = lipsync_detector.compute_sync_score(
            mouth_frames_dir=mouth_output_dir,
            audio_path=audio_output_path
        )
        print(f"✓ Lip-sync score: {lip_sync_score:.4f}")
        print(f"  (Lower score = more suspicious)")
    except Exception as e:
        print(f"⚠ Error in lip-sync analysis: {e}")
        lip_sync_score = None
else:
    print("⚠ No faces to analyze")


## 9. Run Temporal Detection


In [ ]:
# Run temporal detection on face tracks
# Initialize variables if not set
if 'face_count' not in locals():
    face_count = 0
if 'faces_output_dir' not in locals():
    faces_output_dir = None

temporal_mean = 0.5
temporal_max = 0.5

if face_count > 0 and faces_output_dir and Path(faces_output_dir).exists():
    try:
        print("Running temporal detection...")
        # Group faces into tracks
        tracks = temporal_utils.group_faces_into_tracks(faces_output_dir)
        print(f"  Found {len(tracks)} face tracks")
        
        track_scores = []
        for track in tracks:
            track_result = temporal_detector.predict_for_face_track(
                frames_dir=faces_output_dir,
                clip_len=16,
                stride=8
            )
            if track_result["clip_scores"]:
                track_scores.extend(track_result["clip_scores"])
        
        if track_scores:
            temporal_mean = sum(track_scores) / len(track_scores)
            temporal_max = max(track_scores)
            print(f"✓ Temporal analysis complete")
            print(f"  Mean temporal score: {temporal_mean:.4f}")
            print(f"  Max temporal score: {temporal_max:.4f}")
        else:
            print("⚠ No temporal scores computed")
    except Exception as e:
        print(f"⚠ Error in temporal detection: {e}")
else:
    print("⚠ No faces to analyze")


## 10. Aggregate Scores


In [ ]:
# Aggregate all scores
# Initialize variables if not set
if 'detections' not in locals():
    detections = []
if 'lip_sync_score' not in locals():
    lip_sync_score = None
if 'temporal_mean' not in locals():
    temporal_mean = 0.5
if 'temporal_max' not in locals():
    temporal_max = 0.5

aggregation_result = aggregation.aggregate_scores(
    detections=detections,
    lip_sync_score=lip_sync_score,
    temporal_mean=temporal_mean,
    temporal_max=temporal_max
)

print("Aggregated Scores:")
print(f"  Total faces: {aggregation_result['total_faces']}")
print(f"  Max CNN score: {aggregation_result['max_score']:.4f}")
print(f"  Mean CNN score: {aggregation_result['mean_score']:.4f}")
print(f"  Frequency score: {aggregation_result['frequency_score']:.4f}")
print(f"  Lip-sync score: {aggregation_result.get('lip_sync_score', 'N/A')}")
print(f"  Temporal mean: {aggregation_result['temporal_mean']:.4f}")
print(f"  Temporal max: {aggregation_result['temporal_max']:.4f}")

# Visualize aggregated scores
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Score comparison
scores_data = {
    'CNN Max': aggregation_result['max_score'],
    'CNN Mean': aggregation_result['mean_score'],
    'Frequency': aggregation_result['frequency_score'],
    'Temporal Max': aggregation_result['temporal_max'],
    'Temporal Mean': aggregation_result['temporal_mean']
}

if lip_sync_score is not None:
    scores_data['Lip-Sync (inverted)'] = 1.0 - lip_sync_score

axes[0].bar(scores_data.keys(), scores_data.values(), color='steelblue', alpha=0.7)
axes[0].axhline(y=0.5, color='r', linestyle='--', label='Threshold')
axes[0].set_ylabel('Score')
axes[0].set_title('Detector Scores Comparison')
axes[0].set_ylim([0, 1])
axes[0].legend()
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(True, alpha=0.3)

# CNN scores distribution
cnn_scores = [d['fake_score'] for d in detections]
axes[1].hist(cnn_scores, bins=20, edgecolor='black', alpha=0.7, color='coral')
axes[1].axvline(x=0.5, color='r', linestyle='--', label='Threshold')
axes[1].set_xlabel('Fake Score')
axes[1].set_ylabel('Frequency')
axes[1].set_title('CNN Scores Distribution')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 11. Ensemble Fusion


In [ ]:
# Decide verdict using aggregation
verdict_result = aggregation.decide_verdict(aggregation_result)

# Apply ensemble combiner for final verdict
ensemble_combiner = ensemble.EnsembleCombiner()
ensemble_result = ensemble_combiner.combine(aggregation_result)

# Update verdict with ensemble results
if verdict_result.get("label") == "LIKELY_AUTHENTIC":
    if ensemble_result["final_score"] < 0.6:
        verdict_result["final_score"] = ensemble_result["final_score"]
        verdict_result["final_label"] = "LIKELY_AUTHENTIC"
    else:
        verdict_result["final_score"] = ensemble_result["final_score"]
        verdict_result["final_label"] = ensemble_result["final_label"]
        verdict_result["confidence"] = min(verdict_result.get("confidence", 0.5), 0.6)
else:
    verdict_result["final_score"] = ensemble_result["final_score"]
    verdict_result["final_label"] = ensemble_result["final_label"]

print("\n" + "="*60)
print("FINAL VERDICT")
print("="*60)
print(f"Label: {verdict_result['final_label']}")
print(f"Score: {verdict_result['final_score']:.4f}")
print(f"Confidence: {verdict_result.get('confidence', 0.0):.2%}")
print(f"Reasons: {', '.join(verdict_result.get('reason', []))}")
print("="*60)

# Visualize verdict
fig, ax = plt.subplots(figsize=(8, 6))

label = verdict_result['final_label']
score = verdict_result['final_score']

# Color based on verdict
if 'MANIPULATED' in label:
    color = 'red'
elif 'AUTHENTIC' in label:
    color = 'green'
else:
    color = 'orange'

ax.barh([0], [score], color=color, alpha=0.7, height=0.5)
ax.axvline(x=0.5, color='black', linestyle='--', linewidth=2, label='Threshold')
ax.set_xlim([0, 1])
ax.set_xlabel('Manipulation Score', fontsize=12)
ax.set_title(f'Final Verdict: {label}', fontsize=14, fontweight='bold')
ax.text(score, 0, f'{score:.3f}', ha='center', va='center', fontsize=14, fontweight='bold')
ax.set_yticks([])
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


## 12. Generate Abnormality Report


In [ ]:
# Generate abnormality report
# Initialize variables if not set
if 'face_count' not in locals():
    face_count = 0
if 'faces_output_dir' not in locals():
    faces_output_dir = None
if 'detections' not in locals():
    detections = []
if 'temporal_mean' not in locals():
    temporal_mean = 0.5
if 'temporal_max' not in locals():
    temporal_max = 0.5
if 'lip_sync_score' not in locals():
    lip_sync_score = None
if 'aggregation_result' not in locals():
    aggregation_result = {}

abnormality_report = None
technique_report = None

if face_count > 0 and faces_output_dir and Path(faces_output_dir).exists():
    try:
        print("Generating abnormality report...")
        abnormality_analyzer = AbnormalityAnalyzer()
        abnormality_report = abnormality_analyzer.generate_abnormality_report(
            faces_dir=faces_output_dir,
            detections=detections,
            temporal_mean=temporal_mean,
            temporal_max=temporal_max,
            lip_sync_score=lip_sync_score
        )
        
        print(f"✓ Abnormality report generated")
        print(f"  Spatial artifacts: {len(abnormality_report.get('spatial_artifacts', []))}")
        
        # Generate technique identification
        print("\nIdentifying deepfake creation technique...")
        technique_identifier = TechniqueIdentifier()
        technique_report = technique_identifier.generate_technique_report(
            abnormality_report=abnormality_report,
            aggregation=aggregation_result
        )
        
        if technique_report.get("primary_technique"):
            primary = technique_report['primary_technique']
            print(f"✓ Primary technique: {primary['name']}")
            print(f"  Confidence: {primary['confidence']:.1%}")
        else:
            print("⚠ No specific technique identified")
            
    except Exception as e:
        print(f"⚠ Error generating reports: {e}")
        import traceback
        traceback.print_exc()
else:
    print("⚠ No faces to analyze")


## 13. Complete Results Summary


In [ ]:
# Generate complete result
# Initialize variables if not set
if 'job_id' not in locals() or job_id is None:
    job_id = str(uuid.uuid4()) if 'uuid' in dir() else "test"
if 'video_path' not in locals():
    video_path = "storage/uploads/your_video.mp4"
if 'frame_count' not in locals():
    frame_count = 0
if 'detections' not in locals():
    detections = []
if 'lip_sync_score' not in locals():
    lip_sync_score = None
if 'temporal_mean' not in locals():
    temporal_mean = 0.5
if 'temporal_max' not in locals():
    temporal_max = 0.5
if 'abnormality_report' not in locals():
    abnormality_report = None
if 'technique_report' not in locals():
    technique_report = None
if 'verdict_result' not in locals():
    verdict_result = {"final_label": "INCONCLUSIVE", "final_score": 0.5, "confidence": 0.0}
if 'results_dir' not in locals():
    results_dir = os.path.abspath("results")

result = aggregation.generate_result(
    job_id=job_id,
    video_path=video_path,
    frames=frame_count,
    detections=detections,
    lip_sync_score=lip_sync_score,
    temporal_mean=temporal_mean,
    temporal_max=temporal_max,
    abnormality_report=abnormality_report,
    technique_report=technique_report
)

# Update with ensemble verdict
result["verdict"] = verdict_result

# Display summary
print("\n" + "="*60)
print("COMPLETE RESULTS SUMMARY")
print("="*60)
print(f"Job ID: {job_id}")
print(f"Video: {video_path}")
print(f"Frames extracted: {result['frames']}")
print(f"Faces detected: {result['faces']}")
print(f"\nFinal Verdict: {result['verdict']['final_label']}")
print(f"Final Score: {result['verdict']['final_score']:.4f}")
print(f"Confidence: {result['verdict'].get('confidence', 0.0):.2%}")
print("="*60)

# Save result to JSON
result_file = Path(results_dir) / f"{job_id}.json"
result_file.parent.mkdir(parents=True, exist_ok=True)
with open(result_file, 'w', encoding='utf-8') as f:
    json.dump(result, f, indent=2, ensure_ascii=False)
print(f"\n✓ Results saved to: {result_file}")

# Display top suspicious faces
if detections:
    print("\nTop 5 Most Suspicious Faces:")
    for i, det in enumerate(detections[:5], 1):
        print(f"  {i}. Frame {det['frame']}: Score = {det['fake_score']:.4f}")

# Pretty print JSON result
print("\n" + "="*60)
print("Full Result JSON:")
print("="*60)
result_json = json.dumps(result, indent=2, ensure_ascii=False)
if len(result_json) > 2000:
    print(result_json[:2000] + "...")
else:
    print(result_json)


## 14. Visualization of Results


In [ ]:
# Create comprehensive visualization
# Initialize variables if not set
if 'detections' not in locals():
    detections = []
if 'aggregation_result' not in locals():
    aggregation_result = {'max_score': 0.5, 'frequency_score': 0.5, 'temporal_max': 0.5}
if 'lip_sync_score' not in locals():
    lip_sync_score = None
if 'verdict_result' not in locals():
    verdict_result = {"final_label": "INCONCLUSIVE", "final_score": 0.5}
if 'face_count' not in locals():
    face_count = 0
if 'faces_output_dir' not in locals():
    faces_output_dir = None

fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# 1. Score timeline
ax1 = fig.add_subplot(gs[0, :])
if detections:
    frames = [d['frame'] for d in detections]
    scores = [d['fake_score'] for d in detections]
    ax1.plot(frames, scores, 'o-', color='steelblue', linewidth=2, markersize=6)
    ax1.axhline(y=0.5, color='r', linestyle='--', linewidth=2, label='Threshold')
    ax1.set_xlabel('Frame Number')
    ax1.set_ylabel('Fake Score')
    ax1.set_title('CNN Fake Scores Over Time', fontsize=12, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

# 2. Detector comparison
ax2 = fig.add_subplot(gs[1, 0])
detector_names = ['CNN Max', 'Frequency', 'Temporal Max', 'Lip-Sync']
detector_scores = [
    aggregation_result['max_score'],
    aggregation_result['frequency_score'],
    aggregation_result['temporal_max'],
    1.0 - lip_sync_score if lip_sync_score else 0.5
]
colors = ['red' if s > 0.5 else 'green' for s in detector_scores]
ax2.barh(detector_names, detector_scores, color=colors, alpha=0.7)
ax2.axvline(x=0.5, color='black', linestyle='--', linewidth=1)
ax2.set_xlim([0, 1])
ax2.set_xlabel('Score')
ax2.set_title('Detector Scores', fontsize=11, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='x')

# 3. Score distribution
ax3 = fig.add_subplot(gs[1, 1])
if detections:
    scores = [d['fake_score'] for d in detections]
    ax3.hist(scores, bins=15, edgecolor='black', alpha=0.7, color='coral')
    ax3.axvline(x=0.5, color='r', linestyle='--', linewidth=2, label='Threshold')
    ax3.set_xlabel('Fake Score')
    ax3.set_ylabel('Frequency')
    ax3.set_title('Score Distribution', fontsize=11, fontweight='bold')
    ax3.legend()
    ax3.grid(True, alpha=0.3)

# 4. Final verdict
ax4 = fig.add_subplot(gs[1, 2])
label = verdict_result['final_label']
score = verdict_result['final_score']
if 'MANIPULATED' in label:
    verdict_color = 'red'
elif 'AUTHENTIC' in label:
    verdict_color = 'green'
else:
    verdict_color = 'orange'
ax4.barh([0], [score], color=verdict_color, alpha=0.8, height=0.3)
ax4.axvline(x=0.5, color='black', linestyle='--', linewidth=2)
ax4.set_xlim([0, 1])
ax4.set_xlabel('Final Score')
ax4.set_title(f'Final Verdict\n{label}', fontsize=11, fontweight='bold')
ax4.text(score, 0, f'{score:.3f}', ha='center', va='center', fontsize=12, fontweight='bold')
ax4.set_yticks([])
ax4.grid(True, alpha=0.3, axis='x')

# 5. Sample faces with scores
if detections and face_count > 0:
    # Show top 3 most suspicious faces
    top_faces = detections[:3]
    face_files = sorted(Path(faces_output_dir).glob("face_*.jpg"))
    
    for idx, det in enumerate(top_faces):
        # Find corresponding face file
        face_file = next((f for f in face_files if f.name == det['face_file']), None)
        if face_file:
            img = Image.open(face_file)
            ax5_sub = fig.add_subplot(gs[2, idx])
            ax5_sub.imshow(img)
            ax5_sub.set_title(f"Frame {det['frame']}\nScore: {det['fake_score']:.3f}", 
                            fontsize=10, fontweight='bold',
                            color='red' if det['fake_score'] > 0.5 else 'green')
            ax5_sub.axis('off')

plt.suptitle('Deepfake Detection Analysis Report', fontsize=16, fontweight='bold', y=0.995)
plt.show()

print("\n✓ Analysis complete!")
